In [40]:
import sys
sys.path.append("..") # this adds to path parent directory in order to import utils file
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import random
from tqdm import tqdm
import numpy as np
from sklearn import metrics
from IPython.display import clear_output, Image, display
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.decomposition import PCA
import hyperopt
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample
import time
import pickle
import umap
import sys, traceback, os

## Import custom utils
import gmm_model
import utils
import gmm_utils
import hyperopt_utils

In [41]:
%load_ext autoreload
import importlib
importlib.reload(gmm_utils)
importlib.reload(hyperopt_utils)
importlib.reload(utils)
importlib.reload(gmm_model)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
elbowAnalysis
silhouetteAnalyis
elbowAnalysis
silhouetteAnalyis


<module 'gmm_model' from 'C:\\workspace\\ml\\single-cell-sota\\gmm\\gmm_model.py'>

In [32]:
# Check run louvain method
dataset = 'brainCIDR'
df, truth = utils.loadData(dataset)
y = truth.clusters

params = {'dataset': 'brainCIDR', 
          'log': False, 
          'minCellsPerGene': 2, 
          'minGeneDispersion': 1.36, 
          'nb_clusters': 9, 
          'pca_comp': 10, 
          'scaler': 'none'}

In [33]:
for k,v in params.items():
    print(f'self.{k} = {k}', end='\n')
#     print(f'{k} = {v},', end='\n')

self.dataset = dataset
self.log = log
self.minCellsPerGene = minCellsPerGene
self.minGeneDispersion = minGeneDispersion
self.nb_clusters = nb_clusters
self.pca_comp = pca_comp
self.scaler = scaler


In [34]:
model = gmm_model.GMMModel(**params)
model.fitPredict(df)
model.evaluate(y)
# model.runAll()

C:\Users\ciort\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\ciort\Anaconda3\lib\site-packages\sklearn\metrics\cluster\unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances
C:\Users\ciort\Anaconda3\lib\site-packages\sklearn\metrics\cluster\unsupervised.py:342: RuntimeWarning: invalid value encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


{'scaler': 'none',
 'pca_comp': 10,
 'nb_clusters': 9,
 'minGeneDispersion': 1.36,
 'minCellsPerGene': 2,
 'log': False,
 'dataset': 'brainCIDR',
 '_rand_index': 0.7375256528006889,
 '_homogeneity_score': 0.695712157793332,
 '_purity_score': 0.8071428571428572,
 '_adjusted_mutual_info_score': 0.6838563024306835,
 '_fowlkes_mallows_score': 0.7957573448160301,
 '_silhouette_score': 0.07464571196091026,
 '_calinski_harabaz_score': 29.81019852672167,
 '_davies_bouldin_score': 2.4445470993486436}

In [42]:
from sklearn.model_selection import cross_val_score
model = gmm_model.GMMModel(**params)
cross_val_score(model, df, y, scoring = model.scorer, cv = 2)

fit original X (210, 22085)
fit pca X (210, 17115)
scorer (210, 22085), (210,)
predict original X (210, 22085)
predict prepoc X (210, 16151)


ValueError: operands could not be broadcast together with shapes (210,16151) (17115,) 

In [38]:
params

{'dataset': 'brainCIDR',
 'log': False,
 'minCellsPerGene': 2,
 'minGeneDispersion': 1.36,
 'nb_clusters': 9,
 'pca_comp': 10,
 'scaler': 'none'}

## Hyperopt utils

In [3]:
method = 'gmm'
def runHyperopt(trialsFile, resultsFile, space, max_evals = 2, restart = False):
    # Define function to optimise
    def evaluateGMM(args):
        try:
            resultDict, _ = gmm_utils.runGMM(args)
            if os.path.isfile(resultsFile):
                results = pd.read_pickle(resultsFile)

                newDf = pd.DataFrame.from_dict(resultDict, orient='index').T
                results = results.append(newDf)
            else:
                results = pd.DataFrame.from_dict(resultDict, orient='index').T
            results.to_pickle(resultsFile)
        except:
            traceback.print_exc(file=sys.stdout)
            return { 'status' : hyperopt.STATUS_FAIL}

        print(f'>> Result: {resultDict["_rand_index"]}')
        ret = {
            'loss' : -resultDict['_rand_index']
            ,'status' : STATUS_OK
            ,'eval_time' : time.time()        
        }
        return ret

    trials = hyperopt_utils.getTrials(trialsFile ,restart = restart )
    evals_per_epoch = 10
    for e in range(len(trials), max_evals, evals_per_epoch):
        best = fmin(evaluateGMM
                    ,space
                    ,algo=tpe.suggest 
                    ,max_evals= e + evals_per_epoch
                    ,trials=trials)
        print('Index ', e)
#         pickle.dump(trials, open(trialsFile, 'wb'))
    return trials

## Brain CIDR

In [ ]:
dataset = 'brainCIDR'
trialsFile = f'data/{dataset}_gmm_trials.pkl'
resultsFile = f'data/{dataset}_gmm_results.pkl'
space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 3, 300,1))
    ,'nb_clusters' : scope.int(hp.quniform('nb_clusters', 3, 15, 1))
}



In [ ]:
# Smoke testing
# params = {
#     'dataset' : 'brainCIDR'
#     ,'minCellsPerGene':2
#     ,'minGeneDispersion':0.5
#     ,'log' : True
#     ,'scaler' : 'standardScaleCells'
#     ,'pca_comp' : 10
#     ,'nb_clusters' : 10
# }

# params = {'dataset': 'brainCIDR', 
#           'log': False, 
#           'minCellsPerGene': 2, 
#           'minGeneDispersion': 1.3643019957416587, 
#           'nb_clusters': 9, 
#           'pca_comp': 246, 
#           'scaler': 'none'}
# gmm_utils.runGMM(params)

In [ ]:
trials = runHyperopt(trialsFile, resultsFile, space, max_evals = 500, restart = True);

In [ ]:
# gmm_utils.plotBestPrediction(summaryDf, dataset)
resultsDf = pd.read_pickle(resultsFile)
display(resultsDf.sort_values(by='_rand_index', ascending = False).head(3))
gmm_utils.plotCorrelation(resultsDf, name=dataset)

## PancreaticIsletCIDR

In [ ]:
dataset= 'pancreaticIsletCIDR'
df, truth = utils.loadData(dataset)
print(df.shape, truth.shape, truth.clusters.unique())
trialsFile = f'data/{dataset}_gmm_trials.pkl'
resultsFile = f'data/{dataset}_gmm_results.pkl'

In [ ]:

space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 3, 300,1))
    ,'nb_clusters' : scope.int(hp.quniform('nb_clusters', 3, 15, 1))
}



In [ ]:
trials = runHyperopt(trialsFile, resultsFile, space, max_evals = 200, restart = True);

In [ ]:
resultsDf = pd.read_pickle(resultsFile)
display(resultsDf.sort_values(by='_rand_index', ascending = False).head(3))
gmm_utils.plotCorrelation(resultsDf, name=dataset)

## Deng

In [ ]:
dataset= 'deng'
trialsFile = f'data/{dataset}_gmm_trials.pkl'
resultsFile = f'data/{dataset}_gmm_results.pkl'
df, truth = gmm_utils.loadData(dataset)
# umap2D = gmm_utils.getUmap(df, pca_comp = 10)
print(df.shape, truth.shape, truth.clusters.unique())

In [ ]:

space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 3, 300,1))
    ,'nb_clusters' : scope.int(hp.quniform('nb_clusters', 3, 15, 1))
}



In [ ]:
trials = runHyperopt(trialsFile, resultsFile, space, max_evals = 500, restart = True);

In [ ]:
resultsDf = pd.read_pickle(resultsFile)
display(resultsDf.sort_values(by='_rand_index', ascending = False).head(3))
gmm_utils.plotCorrelation(resultsDf, name=dataset)

# sce10x_qc

In [ ]:
dataset= 'sce10x_qc'
df, truth = utils.loadData(dataset)
print(df.shape, truth.shape, truth.clusters.unique())
trialsFile = f'data/{dataset}_{method}_trials.pkl'
resultsFile = f'data/{dataset}_{method}_results.pkl'

In [ ]:
space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 3, 300,1))
    ,'nb_clusters' : scope.int(hp.quniform('nb_clusters', 3, 15, 1))
}

trials = runHyperopt(trialsFile, resultsFile, space, max_evals = 500, restart = True);

# sce2_qc

In [ ]:
dataset= 'sce2_qc'
df, truth = utils.loadData(dataset)
print(df.shape, truth.shape, truth.clusters.unique())
trialsFile = f'data/{dataset}_{method}_trials.pkl'
resultsFile = f'data/{dataset}_{method}_results.pkl'

In [ ]:
space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 3, 300,1))
    ,'nb_clusters' : scope.int(hp.quniform('nb_clusters', 3, 15, 1))
}

trials = runHyperopt(trialsFile, resultsFile, space, max_evals = 500, restart = True);

# sce8_qc

In [ ]:
dataset= 'sce8_qc'
df, truth = utils.loadData(dataset)
print(df.shape, truth.shape, truth.clusters.unique())
trialsFile = f'data/{dataset}_{method}_trials.pkl'
resultsFile = f'data/{dataset}_{method}_results.pkl'

In [ ]:
space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 3, 300,1))
    ,'nb_clusters' : scope.int(hp.quniform('nb_clusters', 3, 15, 1))
}

trials = runHyperopt(trialsFile, resultsFile, space, max_evals = 500, restart = True);

In [ ]:
bestBic, bestAic, bestSil = optimalNbClustersGMM(pc, params['min_clusters'], params['max_clusters'], 2)

In [ ]:
bestBic, bestAic, bestSil

In [ ]:
for n_clust in n_clusters:
    model = GaussianMixture(n_clust, covariance_type ='full', random_state = 0).fit(pc)
    clusters = model.predict(pc)
    score = adjusted_rand_score(truth.clusters.tolist(), clusters)
    print(f"For {n_clust} clusters, score : {score}")

## Evaluation

In [ ]:
model = GaussianMixture(8, covariance_type ='full', random_state = 0).fit(pc)
clusters = model.predict(pc)
score = adjusted_rand_score(truth.clusters.tolist(), clusters)
score

In [ ]:
truth.clusters.value_counts().shape

In [ ]:
dataset = 'brainCIDR'
df, truth = gmm_utils.loadData(dataset)
umap2D = gmm_utils.getUmap(df, pca_comp = 10)
print(df.shape, truth.shape, truth.clusters.unique())
params={}
params['dataset'] = 'brainCIDR'
params['minCellsPerGene'] = 0
params['minGeneDispersion'] = 0
params['log'] = True # True, False
params['scaler'] = 'standardScaleCells'# 
params['pca_comp'] = 10 #range (3, 300)
params['nb_clusters'] =8 #3 -15
gmm_utils.run(params);